In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.0 contractions torch==1.3.1
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch import nn, cuda
import pandas as pd
from sklearn import metrics
import seaborn
import transformers
import torch
import nltk
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from torch import cuda, nn
import torch
import pandas as pd
import re
import string
#For plots
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
device = 'cuda' if cuda.is_available() else 'cpu'

     |████████████████████████████████| 1.4MB 8.0MB/s 
     |████████████████████████████████| 734.6MB 21kB/s 
     |████████████████████████████████| 901kB 45.4MB/s 
     |████████████████████████████████| 2.9MB 48.2MB/s 
     |████████████████████████████████| 286kB 52.1MB/s 
     |████████████████████████████████| 327kB 54.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85385 sha256=b08f7a4021092045b91f12b48bd7dceec316120fa85bc6021b04fc7137f9b1af
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Some Utility Functions

In [3]:
# Remove Stop words
def remove_Sw(all_words, stops):
  sent = []
  for word in all_words: 
      if word not in stops:
          sent.append(word)
  return " ".join(sent)

# Remove digits
def removeDigits(string):
  string = string.lower().replace("\n", "")
  return ''.join(i for i in string if not i.isdigit())

# Get length of list
def getLength(string):
  return len(string.split(" "))

# Remove empty string from list
def removeEmpty(li):
  while("" in li) :
      li.remove("")
  while(" " in li) :
      li.remove(" ")
  return li

# Remove other punctuations
def removePunct(sent):
  updt = sent.replace(':','').replace(';','').replace("'",'').replace('"','').replace('(','').replace(')','').replace('_','').replace('/','').replace('__','')
  return updt

# Assigning labels
def labels(samples):
  li = []
  for each in samples:
    each = each.strip()
    if "," in each:
      li.append(1)
    elif "?" in each:
      li.append(2)
    elif "." in each:
      li.append(3)
    elif "!" in each:
      li.append(4)  
    else: li.append(0)
  return li

# Aligning labels w.r.t tokens
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(list(examples['new_tokens']), truncation=True, is_split_into_words=True, max_length=512, pad_to_max_length=True)
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(0)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else 0)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Converting .txt file to .csv after removing stop words

In [ ]:
with open("/content/drive/MyDrive/tr_ds_ds.txt",'r') as f:
  txt = f.readlines()with open("/content/drive/MyDrive/tr_ds_ds.txt",'r') as f:
  txt = f.readlines()

In [ ]:
data = pd.DataFrame()
data['text'] = txt
data['text'] = data['text'].str.lower()
data

In [ ]:
from nltk.corpus import stopwords
# For Turkish
turk_stops = set(stopwords.words('turkish'))
print(turk_stops)
data["clean_txt"] = data["clean_txt"].apply(lambda x: remove_Sw(x.split(" "), turk_stops))

# For Azerbaijan
aze_stops = set(stopwords.words('azerbaijani'))
print(aze_stops)
data["clean_txt"] = data["clean_txt"].apply(lambda x: remove_Sw(x.split(" "), aze_stops))

In [ ]:
# FROM GTHUB
# stop_words = pd.read_csv("https://raw.githubusercontent.com/InJuxSanct/turkish-stopwords/master/src/lib/stopwords/raw-stopwords.txt", sep=" ", header=None)
# stop_words.columns=["words_list"]
# sw = stop_words['words_list'].to_list()
# data["clean_txt"] = data["text"].apply(lambda x: remove_Sw(x.split(" "), sw))

In [ ]:
data.to_csv("/content/drive/MyDrive/tr_ds_ds.csv")

### Loading .csv file

In [5]:
df = pd.read_csv("/content/drive/MyDrive/punctuation data/aze_ds_ds.csv").sample(5)
df.drop(columns=["text", "Unnamed: 0"], axis=1, inplace=True)
df.head()

,clean_txt
667414,"uğurlara baxmayaraq, curtis ağır depressiyadan..."
421653,ələddin çələbi osmanlı alimi şeyxülislamı. \n
578512,kommuna dəniz səviyyəsindən 752 3464 metr yüks...
589895,ordunun hərbi quruluşu. \n
462221,santiyeqo de kompostela kafedralından götürülm...


In [6]:
# Remove Digits
df["clean_txt"] = df["clean_txt"].apply(lambda x: removeDigits(x))
# Remove punctuations
df["clean_txt"] = df["clean_txt"].apply(lambda x: removePunct(x))
# Splitting into tokens
df["tokens"] = df['clean_txt'].apply(lambda x: x.split(" "))
# Remove EMPTY
df['new_tokens'] = df['tokens'].apply(lambda x: removeEmpty(x))
# Getting length
df['length'] = df['tokens'].apply(lambda x: len(x))
# Create Labels
df['labels'] = df['tokens'].apply(lambda x: labels(x))

In [7]:
df.drop(['length', 'clean_txt'], axis=1, inplace=True)
df.head()

,tokens,new_tokens,labels
667414,"[uğurlara, baxmayaraq,, curtis, ağır, depressi...","[uğurlara, baxmayaraq,, curtis, ağır, depressi...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 3]"
421653,"[ələddin, çələbi, osmanlı, alimi, şeyxülislamı.]","[ələddin, çələbi, osmanlı, alimi, şeyxülislamı.]","[0, 0, 0, 0, 3]"
578512,"[kommuna, dəniz, səviyyəsindən, metr, yüksəkli...","[kommuna, dəniz, səviyyəsindən, metr, yüksəkli...","[0, 0, 0, 0, 0, 3]"
589895,"[ordunun, hərbi, quruluşu.]","[ordunun, hərbi, quruluşu.]","[0, 0, 3]"
462221,"[santiyeqo, de, kompostela, kafedralından, göt...","[santiyeqo, de, kompostela, kafedralından, göt...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


### Dataloader Class

In [8]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, device):
        # super(CustomDataset, self).__init__(dataframe, label)
        self.dataframe = dataframe
        self.ids = self.dataframe['input_ids'].values.tolist()
        self.mask = self.dataframe['attention_mask'].values.tolist()
        self.targets = self.dataframe['labels'].values.tolist()
       
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):

        ids = self.ids[index]
        mask = self.mask[index]
        t_value = self.targets[index]       #index by iteration
        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device, dtype = torch.long),
            'mask': torch.tensor(mask, dtype=torch.long).to(device, dtype = torch.long),
            'targets': torch.tensor(t_value, dtype=torch.long).to(device)
        }

### Model Class and Loading model

In [9]:
!pip install dotmap
# import logging
import sys
import numpy as np
from abc import abstractmethod
import torch
import torch.nn as nn


# handler = logging.StreamHandler(sys.stdout)
# handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-9s %(message)s'))

# log = logging.getLogger(__name__)
# log.setLevel(logging.INFO)
# log.addHandler(handler)


class BaseModel(nn.Module):
    """
    Base class for all torch models
    """
    def __init__(self, config):
        super().__init__()
        self._config = config

    @abstractmethod
    def forward(self, *inputs):
        """
        Forward pass logic
        :return: Model output
        """
        raise NotImplementedError

    def __str__(self):
        """
        Model prints with number of trainable parameters
        """
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return super().__str__() + '\nTrainable parameters: {}'.format(params)

    def save_model(self, optimizer, epoch):
        log.info("Saving model...")
        torch.save(self.state_dict(), f'saved/models/{self._config.model.name}_{str(epoch)}.pth')
        torch.save(optimizer.state_dict(), f'saved/models/{self._config.model.name}_{str(epoch)}_optimizer_state.pth')

    def load_model(self, optimizer, epoch, model_name):
        log.info("Loading model...")
        self.load_state_dict(torch.load(f'saved/models/{model_name}_{str(epoch)}.pth'))
        optimizer.load_state_dict(torch.load(f'saved/models/{model_name}_{str(epoch)}_optimizer_state.pth'))

from typing import Any

import torch
from torch import nn
import torch.nn.functional as F


from transformers import AutoTokenizer, AutoModel


class BertPunctuator(BaseModel):
    def __init__(self, config):
        super().__init__(config)
        self.base = AutoModel.from_pretrained(self._config.model.load_model_repo)

        if not self._config.trainer.train_bert:
            for param in self.base.parameters():
                param.requires_grad = False

        self.classifier = Classifier(self._config)

    def forward(self, x, y):
        if self._config.trainer.train_bert:
            embedding, _ = self.base(input_ids=x, attention_mask=y, return_dict=False)
        else:
            with torch.no_grad():
                embedding, _ = self.base(input_ids=x, attention_mask=y, return_dict=False)

        output = self.classifier(embedding)
        output = F.log_softmax(output, dim=-1)
        return output

    def train(self, mode=True):
        if mode:
            if self._config.trainer.train_bert:
                self.base.train()
            else:
                self.base.eval()
            self.classifier.train()
        else:
            self.base.eval()
            self.classifier.eval()
        return self

    def eval(self):
        self.train(False)
        return self


class Classifier(BaseModel):
    def __init__(self, config):
        super().__init__(None)
        self.dropout1 = nn.Dropout(0.1)
        self.linear1 = nn.Linear(config.model.bert_output_dim, config.model.linear_hidden_dim)
        self.activation = nn.ReLU()
        self.dropout2 = nn.Dropout(0.1)
        self.linear2 = nn.Linear(config.model.linear_hidden_dim, config.model.num_classes)

    def forward(self, x):
        x = self.dropout1(x)
        x = self.activation(self.linear1(x))
        x = self.dropout2(x)
        x = self.linear2(x)
        return x


import yaml
from dotmap import DotMap
def get_config_from_yaml(yaml_file):
    with open(yaml_file, 'r') as config_file:
        config_yaml = yaml.load(config_file)
    # Using DotMap we will be able to reference nested parameters via attribute such as x.y instead of x['y']
    config = DotMap(config_yaml)
    return config

In [10]:
config = get_config_from_yaml('/content/drive/MyDrive/punctuation data/model.yaml')
model = BertPunctuator(config)
model.to(device)

BertPunctuator(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### Aligning labels according to tokens

In [11]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
inputs = tokenize_and_align_labels(df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
df_ = pd.DataFrame.from_dict(inputs, orient='index')
df_ = df_.T
df_.head()

,input_ids,attention_mask,labels
0,"[101, 1057, 27390, 8017, 2050, 8670, 2595, 278...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
1,"[101, 1114, 2140, 29681, 18277, 1039, 29681, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[101, 12849, 7382, 9521, 1040, 29681, 3490, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[101, 2030, 27584, 4609, 1044, 29681, 15185, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, ..."
4,"[101, 15548, 28008, 2063, 4160, 2080, 2139, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
L = ['input_ids','attention_mask','labels']
for z in L:
  for i in df_[z]:
    if len(i) != 512:
      print('error')

### Loading Data

In [15]:
testing_set = CustomDataset(df_, device)
print(f'Dataset size {testing_set.__len__()}')
testing_loader = DataLoader(testing_set, batch_size = testing_set.__len__(), num_workers=0, shuffle = True)

Dataset size 5


In [16]:
LEARNING_RATE = 0.00003
criterion = nn.BCEWithLogitsLoss()     # we can use it for cross entropy loss. in case of focal loss we don't need this
optimizer = torch.optim.SGD(params =  model.parameters(), lr=LEARNING_RATE, momentum=0.9)

### Evaluation script

In [17]:
softm = torch.nn.Softmax(dim=1)
def Evaluate(model, testloader, optimizer, model_path):
    t_f1, accuracy, cls_tars, cls_preds = [], [], [], []
    with torch.no_grad():
      model.load_state_dict(torch.load(model_path))
      model.eval()
      pbar = tqdm(enumerate(testloader, 0)) 
      for batch, data in pbar:
          pbar.set_description(f'Testing batch no {batch}')
          id = data['ids']
          mask = data['mask']
          targets = torch.nn.functional.one_hot(data['targets'], 5)
          one_hot = targets.to(device, dtype = torch.float)
          optimizer.zero_grad()
          out = model(id, mask)
              
          preds = torch.argmax(softm(out),dim=2).cpu().detach().numpy()
          acc = accuracy_score(data['targets'].flatten().cpu().numpy(), preds.flatten())
          accuracy.append(acc)
          f1 = f1_score(data['targets'].flatten().cpu().numpy(), preds.flatten(), average='weighted')
          t_f1.append(f1)
          cls_tars.extend(data['targets'].cpu().numpy())
          cls_preds.extend(preds)

      final_vacc = np.mean(accuracy)
      final_f1 = np.mean(t_f1)        

      print(f'\nTest Accuracy: {100*final_vacc} \nTest F1 Score: {final_f1}')
      print(f'Precision: {precision_score(np.array(cls_tars).flatten(), np.array(cls_preds).flatten(), zero_division=1, average="weighted")}')
      print(f'Recall: {recall_score(np.array(cls_tars).flatten(), np.array(cls_preds).flatten(), zero_division=1, average="weighted")}\n')


In [18]:
Evaluate(model, testing_loader, optimizer, model_path='/content/drive/MyDrive/punctuation data/Azerbaijan/Aze2_Bert.pt')

Testing batch no 0: : 1it [00:00,  1.90it/s]


Test Accuracy: 91.2109375 
Test F1 Score: 0.9421074759781455
Precision: 0.9839190237872601
Recall: 0.912109375



## Testing and Generating back a string

In [23]:
model.load_state_dict(torch.load('./drive/MyDrive/punctuation data/Azerbaijan/Aze2_Bert.pt'))

<All keys matched successfully>

In [24]:
x = torch.tensor(df_['input_ids'][0]).unsqueeze(0).to(device)
y = torch.tensor(df_['attention_mask'][0]).unsqueeze(0).to(device)
x.shape, y.shape

(torch.Size([1, 512]), torch.Size([1, 512]))

In [25]:
output = model(x,y)
output.shape

torch.Size([1, 512, 5])

In [26]:
output = output.argmax(-1)[0].detach().cpu()
output

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
id2target = {-1: 0,
              9: 3, # .
              60: 2, # ?
              15: 1,  # ,
              30: 4,  # !
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}

In [28]:
generated_output = []

for token_id, target in zip(df_['input_ids'][0], output):
    generated_output.append(token_id)
    
    token = tokenizer._convert_id_to_token(token_id)
    if token == '<pad>':
        break
    if target > 0:
        target_id = target2id[target.item()]
        target_token = tokenizer._convert_id_to_token(target_id)
        
        generated_output.append(target_id)
    else:
        target_token = ""
    print(token, target_token)

[CLS] 
u 
##gur 
##lar 
##a 
ba 
##x [unused14]
##may [unused14]
##ara [unused14]
##q [unused14]
, [unused14]
curtis 
ag 
##ı 
##r 
de 
##press [unused14]
##iya [unused14]
##dan [unused14]
, [unused14]
ep 
##ile 
##ps 
##iya 
bo 
##san 
##ma 
s 
##ə 
##x 
##si 
problem 
##l 
##ə 
##rd 
##ə 
##n 
ə 
##zi 
##y 
##y 
##ə 
##t 
c 
##ə 
##kir [unused8]
##di [unused8]
. [unused8]
[SEP] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 
[PAD] 


In [35]:
result = tokenizer.decode(generated_output[1:])
result

'ugurlara bax [unused14]may [unused14]ara [unused14]q [unused14], [unused14] curtis agır depress [unused14]iya [unused14]dan [unused14], [unused14] epilepsiya bosanma səxsi problemlərdən əziyyət cəkir [unused8]di [unused8]. [unused8] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [36]:
result.replace(' [unused14]','').replace('[PAD]','').replace('[unused8]','').replace('[SEP]','')

'ugurlara baxmayaraq, curtis agır depressiyadan, epilepsiya bosanma səxsi problemlərdən əziyyət cəkir di .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 '

In [37]:
original_sentence = " ".join(df['new_tokens'][667414])
original_sentence

'uğurlara baxmayaraq, curtis ağır depressiyadan, epilepsiya boşanma şəxsi problemlərdən əziyyət çəkirdi.'